In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

### Data Source 1

In [2]:
# store CSV into DataFrame
csv_file = "resources/county_level_confirmed_cases.csv"
cases_df = pd.read_csv(csv_file)
cases_df

,last_update,location_type,state,county_name,county_name_long,fips_code,lat,lon,nchs_urbanization,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000,location
0,2020-09-04 00:28:22 UTC,county,Alabama,Autauga,"Autauga, Alabama, US",1001.0,32.539527,-86.644082,Medium metro,55200.0,1349,2443.84,23,41.67,POINT(-86.64408227 32.53952745)
1,2020-09-04 00:28:22 UTC,county,Alabama,Baldwin,"Baldwin, Alabama, US",1003.0,30.727750,-87.722071,Small metro,208107.0,4495,2159.95,40,19.22,POINT(-87.72207058 30.72774991)
2,2020-09-04 00:28:22 UTC,county,Alabama,Barbour,"Barbour, Alabama, US",1005.0,31.868263,-85.387129,Non-core,25782.0,614,2381.51,7,27.15,POINT(-85.3871286 31.868263)
3,2020-09-04 00:28:22 UTC,county,Alabama,Bibb,"Bibb, Alabama, US",1007.0,32.996421,-87.125115,Large fringe metro,22527.0,542,2406.00,6,26.63,POINT(-87.1251146 32.99642064)
4,2020-09-04 00:28:22 UTC,county,Alabama,Blount,"Blount, Alabama, US",1009.0,33.982109,-86.567906,Large fringe metro,57645.0,1037,1798.94,11,19.08,POINT(-86.56790593 33.98210918)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,2020-09-04 00:28:22 UTC,special_exception,Utah,Central Utah,"Central Utah, Utah, US",NaN,39.372319,-111.575868,NaN,NaN,523,NaN,3,NaN,POINT(-111.5758676 39.37231946)
3260,2020-09-04 00:28:22 UTC,special_exception,Utah,Southeast Utah,"Southeast Utah, Utah, US",NaN,38.996171,-110.701396,NaN,NaN,154,NaN,1,NaN,POINT(-110.7013958 38.99617072)
3261,2020-09-04 00:28:22 UTC,special_exception,Utah,Southwest Utah,"Southwest Utah, Utah, US",NaN,37.854472,-111.441876,NaN,NaN,3664,NaN,27,NaN,POINT(-111.4418764 37.85447192)
3262,2020-09-04 00:28:22 UTC,special_exception,Utah,TriCounty,"TriCounty, Utah, US",NaN,40.124915,-109.517442,NaN,NaN,214,NaN,1,NaN,POINT(-109.5174415 40.12491499)


In [3]:
# create new data with select columns
new_cases_df = cases_df[['last_update', 
                         'fips_code', 
                         'county_name', 
                         'state', 
                         'total_population', 
                         'confirmed', 
                         'confirmed_per_100000', 
                         'deaths', 
                         'deaths_per_100000']
                       ].copy()
new_cases_df.head()

,last_update,fips_code,county_name,state,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,2020-09-04 00:28:22 UTC,1001.0,Autauga,Alabama,55200.0,1349,2443.84,23,41.67
1,2020-09-04 00:28:22 UTC,1003.0,Baldwin,Alabama,208107.0,4495,2159.95,40,19.22
2,2020-09-04 00:28:22 UTC,1005.0,Barbour,Alabama,25782.0,614,2381.51,7,27.15
3,2020-09-04 00:28:22 UTC,1007.0,Bibb,Alabama,22527.0,542,2406.00,6,26.63
4,2020-09-04 00:28:22 UTC,1009.0,Blount,Alabama,57645.0,1037,1798.94,11,19.08


In [4]:
# change column name
new_cases_df.rename(columns = {'state': 'state_name'}, inplace=True)

In [5]:
# identify incomplete rows
new_cases_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3264 entries, 0 to 3263
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   last_update           3264 non-null   object 
 1   fips_code             3254 non-null   float64
 2   county_name           3264 non-null   object 
 3   state_name            3264 non-null   object 
 4   total_population      3192 non-null   float64
 5   confirmed             3264 non-null   int64  
 6   confirmed_per_100000  3192 non-null   float64
 7   deaths                3264 non-null   int64  
 8   deaths_per_100000     3192 non-null   float64
dtypes: float64(4), int64(2), object(3)
memory usage: 229.6+ KB


In [6]:
# drop incomplete rows
new_cases_df.dropna(how='any', inplace=True)

In [7]:
# verify incomplete rows dropped
new_cases_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3192 entries, 0 to 3193
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   last_update           3192 non-null   object 
 1   fips_code             3192 non-null   float64
 2   county_name           3192 non-null   object 
 3   state_name            3192 non-null   object 
 4   total_population      3192 non-null   float64
 5   confirmed             3192 non-null   int64  
 6   confirmed_per_100000  3192 non-null   float64
 7   deaths                3192 non-null   int64  
 8   deaths_per_100000     3192 non-null   float64
dtypes: float64(4), int64(2), object(3)
memory usage: 249.4+ KB


In [8]:
# use dictionary to convert specific column dtype
convert_dict = {'fips_code': int,
                'total_population': int
               }
new_cases_df = new_cases_df.astype(convert_dict) 

In [9]:
# check data types
new_cases_df.dtypes

last_update              object
fips_code                 int32
county_name              object
state_name               object
total_population          int32
confirmed                 int64
confirmed_per_100000    float64
deaths                    int64
deaths_per_100000       float64
dtype: object

In [10]:
# add leading zeros to fips_code
new_cases_df['fips_code']=new_cases_df['fips_code'].apply(lambda x: '{0:0>5}'.format(x))
new_cases_df.head()

,last_update,fips_code,county_name,state_name,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,2020-09-04 00:28:22 UTC,01001,Autauga,Alabama,55200,1349,2443.84,23,41.67
1,2020-09-04 00:28:22 UTC,01003,Baldwin,Alabama,208107,4495,2159.95,40,19.22
2,2020-09-04 00:28:22 UTC,01005,Barbour,Alabama,25782,614,2381.51,7,27.15
3,2020-09-04 00:28:22 UTC,01007,Bibb,Alabama,22527,542,2406.00,6,26.63
4,2020-09-04 00:28:22 UTC,01009,Blount,Alabama,57645,1037,1798.94,11,19.08


### Data Source 2

In [11]:
# store Excel into DataFrame
unemployment_df = pd.read_excel (r'C:\Users\dave\Data_Bootcamp\Homework\ETL-Project\resources\unemployment.xls', sheet_name='Unemployment Med HH Income')
unemployment_df

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0,US,United States,NaN,NaN,NaN,142601667.0,136904680.0,5696987.0,3.995035,...,161389026.0,155102319.0,6286707.0,3.895375,163100055.0,157115247.0,5984808.0,3.669409,61937.0,NaN
1,1000,AL,Alabama,NaN,NaN,NaN,2133223.0,2035594.0,97629.0,4.600000,...,2216627.0,2130845.0,85782.0,3.900000,2241747.0,2174483.0,67264.0,3.000000,49881.0,100.000000
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,21720.0,20846.0,874.0,4.000000,...,26196.0,25261.0,935.0,3.600000,26172.0,25458.0,714.0,2.700000,59338.0,118.959123
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,69533.0,66971.0,2562.0,3.700000,...,95233.0,91809.0,3424.0,3.600000,97328.0,94675.0,2653.0,2.700000,57588.0,115.450773
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,11373.0,10748.0,625.0,5.500000,...,8414.0,7987.0,427.0,5.100000,8537.0,8213.0,324.0,3.800000,34382.0,68.928049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270,72145,PR,"Vega Baja Municipio, PR",1.0,1.0,1.0,19805.0,17607.0,2198.0,11.100000,...,13117.0,11750.0,1367.0,10.400000,13037.0,11791.0,1246.0,9.600000,NaN,NaN
3271,72147,PR,"Vieques Municipio, PR",7.0,12.0,0.0,2441.0,2120.0,321.0,13.200000,...,2718.0,2269.0,449.0,16.500000,2585.0,2406.0,179.0,6.900000,NaN,NaN
3272,72149,PR,"Villalba Municipio, PR",2.0,2.0,1.0,7636.0,6648.0,988.0,12.900000,...,7311.0,6154.0,1157.0,15.800000,7406.0,6231.0,1175.0,15.900000,NaN,NaN
3273,72151,PR,"Yabucoa Municipio, PR",1.0,1.0,1.0,10613.0,9005.0,1608.0,15.200000,...,8762.0,7509.0,1253.0,14.300000,8691.0,7552.0,1139.0,13.100000,NaN,NaN


In [12]:
unemployment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3275 entries, 0 to 3274
Data columns (total 88 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   FIPStxt                                    3275 non-null   int64  
 1   Stabr                                      3275 non-null   object 
 2   area_name                                  3275 non-null   object 
 3   Rural_urban_continuum_code_2013            3219 non-null   float64
 4   Urban_influence_code_2013                  3219 non-null   float64
 5   Metro_2013                                 3222 non-null   float64
 6   Civilian_labor_force_2000                  3270 non-null   float64
 7   Employed_2000                              3270 non-null   float64
 8   Unemployed_2000                            3270 non-null   float64
 9   Unemployment_rate_2000                     3270 non-null   float64
 10  Civilian_labor_force_200

In [13]:
# create new data frame with select columns
new_unemployment_df = unemployment_df[['FIPStxt',
                                       'Civilian_labor_force_2019', 
                                       'Employed_2019', 
                                       'Unemployed_2019', 
                                       'Unemployment_rate_2019', 
                                       'Median_Household_Income_2018', 
                                       'Med_HH_Income_Percent_of_State_Total_2018']
                                     ].copy()
new_unemployment_df.head()

,FIPStxt,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0,163100055.0,157115247.0,5984808.0,3.669409,61937.0,NaN
1,1000,2241747.0,2174483.0,67264.0,3.000000,49881.0,100.000000
2,1001,26172.0,25458.0,714.0,2.700000,59338.0,118.959123
3,1003,97328.0,94675.0,2653.0,2.700000,57588.0,115.450773
4,1005,8537.0,8213.0,324.0,3.800000,34382.0,68.928049


In [14]:
new_unemployment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3275 entries, 0 to 3274
Data columns (total 7 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   FIPStxt                                    3275 non-null   int64  
 1   Civilian_labor_force_2019                  3272 non-null   float64
 2   Employed_2019                              3272 non-null   float64
 3   Unemployed_2019                            3272 non-null   float64
 4   Unemployment_rate_2019                     3272 non-null   float64
 5   Median_Household_Income_2018               3193 non-null   float64
 6   Med_HH_Income_Percent_of_State_Total_2018  3192 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 179.2 KB


In [15]:
# fill all NA's with 0
new_unemployment_df.fillna(value=0, inplace=True)

In [16]:
new_unemployment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3275 entries, 0 to 3274
Data columns (total 7 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   FIPStxt                                    3275 non-null   int64  
 1   Civilian_labor_force_2019                  3275 non-null   float64
 2   Employed_2019                              3275 non-null   float64
 3   Unemployed_2019                            3275 non-null   float64
 4   Unemployment_rate_2019                     3275 non-null   float64
 5   Median_Household_Income_2018               3275 non-null   float64
 6   Med_HH_Income_Percent_of_State_Total_2018  3275 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 179.2 KB


In [17]:
# use dictionary to convert specific columns dtype
convert_dict = {'Civilian_labor_force_2019': int, 
                'Employed_2019': int, 
                'Unemployed_2019': int
               }
new_unemployment_df = new_unemployment_df.astype(convert_dict) 

In [18]:
# verify column dtypes
new_unemployment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3275 entries, 0 to 3274
Data columns (total 7 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   FIPStxt                                    3275 non-null   int64  
 1   Civilian_labor_force_2019                  3275 non-null   int32  
 2   Employed_2019                              3275 non-null   int32  
 3   Unemployed_2019                            3275 non-null   int32  
 4   Unemployment_rate_2019                     3275 non-null   float64
 5   Median_Household_Income_2018               3275 non-null   float64
 6   Med_HH_Income_Percent_of_State_Total_2018  3275 non-null   float64
dtypes: float64(3), int32(3), int64(1)
memory usage: 140.8 KB


In [19]:
# add leading zeros to fips_code
new_unemployment_df['FIPStxt']=new_unemployment_df['FIPStxt'].apply(lambda x: '{0:0>5}'.format(x))
new_unemployment_df.head()

,FIPStxt,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,00000,163100055,157115247,5984808,3.669409,61937.0,0.000000
1,01000,2241747,2174483,67264,3.000000,49881.0,100.000000
2,01001,26172,25458,714,2.700000,59338.0,118.959123
3,01003,97328,94675,2653,2.700000,57588.0,115.450773
4,01005,8537,8213,324,3.800000,34382.0,68.928049


In [20]:
# delete row where FIPS = 00000 
new_unemployment_df.drop(new_unemployment_df[new_unemployment_df['FIPStxt'] == '00000'].index, inplace = True)
new_unemployment_df.head()

,FIPStxt,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
1,01000,2241747,2174483,67264,3.0,49881.0,100.000000
2,01001,26172,25458,714,2.7,59338.0,118.959123
3,01003,97328,94675,2653,2.7,57588.0,115.450773
4,01005,8537,8213,324,3.8,34382.0,68.928049
5,01007,8685,8419,266,3.1,46064.0,92.347788


### Load data to SQL database

In [21]:
# connect to local database
connection_string = "postgres:postgres@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{connection_string}')

In [22]:
# check for tables
engine.table_names()

['cases', 'unemployment']

In [23]:
# use pandas to load csv converted DataFrame into database
new_cases_df.to_sql(name='cases', con=engine, if_exists='append', index=False)

In [24]:
# use pandas to load excel converted DataFrame into database
new_unemployment_df.to_sql(name='unemployment', con=engine, if_exists='append', index=False)

In [25]:
# confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from cases', con=engine).head()

,last_update,fips_code,county_name,state_name,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,2020-09-04 00:28:22+00:00,1001,Autauga,Alabama,55200,1349,2443.84,23,41.67
1,2020-09-04 00:28:22+00:00,1003,Baldwin,Alabama,208107,4495,2159.95,40,19.22
2,2020-09-04 00:28:22+00:00,1005,Barbour,Alabama,25782,614,2381.51,7,27.15
3,2020-09-04 00:28:22+00:00,1007,Bibb,Alabama,22527,542,2406.00,6,26.63
4,2020-09-04 00:28:22+00:00,1009,Blount,Alabama,57645,1037,1798.94,11,19.08


In [26]:
# confirm data has been added by querying the customer_location table
pd.read_sql_query('select * from unemployment', con=engine).head()

,FIPStxt,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,1000,2241747,2174483,67264,3.0,49881.0,100.000000
1,1001,26172,25458,714,2.7,59338.0,118.959120
2,1003,97328,94675,2653,2.7,57588.0,115.450775
3,1005,8537,8213,324,3.8,34382.0,68.928050
4,1007,8685,8419,266,3.1,46064.0,92.347786
